In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTAttnCRF

In [7]:
model = BERTAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [8]:
from modules.train.train import NerLearner

In [9]:
num_epochs = 100

In [18]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [19]:
model.get_n_trainable_params()

890617

In [21]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 47.


INFO:root:
epoch 48, average train epoch loss=0.076331



INFO:root:on epoch 42 by max_f1: 0.891


              precision    recall  f1-score   support

         I_O      0.975     0.988     0.981     48875
       I_LOC      0.883     0.787     0.832      1557
       I_PER      0.952     0.953     0.953      2112
       I_ORG      0.844     0.731     0.783      3865

   micro avg      0.964     0.963     0.964     56409
   macro avg      0.913     0.865     0.887     56409
weighted avg      0.962     0.963     0.963     56409



### Eval

#### Load model

In [4]:
from modules.data import bert_data

In [5]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [6]:
from modules.models.bert_models import BERTAttnCRF

In [7]:
model = BERTAttnCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3)

In [8]:
from modules.train.train import NerLearner

In [10]:
num_epochs = 100

In [11]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTAttnCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [12]:
model.get_n_trainable_params()

890617

In [13]:
learner.load_model()

### Predict

In [14]:
from modules.data.bert_data import get_data_loader_for_predict

In [15]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [16]:
preds = learner.predict(dl)

In [17]:
from sklearn_crfsuite.metrics import flat_classification_report

In [18]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [19]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [20]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [21]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8357    0.7581    0.7950      3865
       I_PER     0.9342    0.9673    0.9505      2112
       I_LOC     0.8879    0.8035    0.8436      1557

   micro avg     0.8764    0.8261    0.8505      7534
   macro avg     0.8859    0.8430    0.8630      7534
weighted avg     0.8741    0.8261    0.8486      7534



In [22]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [23]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7893    0.7108    0.7480      1307
           O     0.9734    0.9823    0.9779     48691
         ORG     0.6611    0.5510    0.6010      1873
         PER     0.8256    0.8237    0.8247      1299

    accuracy                         0.9566     53170
   macro avg     0.8124    0.7670    0.7879     53170
weighted avg     0.9543    0.9566    0.9552     53170

